In [4]:
import math
import random

In [28]:
class Value:
    def __init__(self, data, _children=(), _op=''):
        self.data = data
        self.grad = 0
        self._backward = lambda: None 
        self._prev = set(_children)
        self._op = _op

    def __repr__(self):
        return f'Value({self.data})'
    
    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')
        def _backward():
            self.grad += out.grad
            other.grad += out.grad
        out._backward = _backward
        return out
    
    def __radd__(self, other):
        return self + other

    def __neg__(self):
        return self * -1

    def __sub__(self, other):
        return self + (-other)

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
        return out

    def __rmul__(self, other):  
        return self * other  # a * b == b * a

    def __truediv__(self, other):
        return self * other**-1
    
    def __pow__(self, other):
        assert isinstance(other, (int, float)), 'only support int or float'
        out = Value(self.data**other, (self,), f'**{other}')
        def _backward():
            self.grad += other * self.data**(other - 1) * out.grad
        out._backward = _backward
        return out

    def exp(self):
        out = Value(math.exp(self.data), (self,), 'exp')
        def _backward():
            self.grad += out.data * out.grad
        out._backward = _backward
        return out
    
    def tanh(self):
        e2x = (2 * self).exp()
        return (e2x - 1) / (e2x + 1)
    
    def backward(self):
        topo = []
        visited = set()
        def topo_sort(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    topo_sort(child)
                topo.append(v)

        topo_sort(self)
        self.grad = 1
        for v in reversed(topo):
            v._backward()

In [29]:
a = Value(2.5)
b = Value(-1.5)
c = Value(3.0)
e = Value(4.0)
d = a + (b * c * e)

d.backward()

In [30]:
d.grad, c.grad, b.grad, a.grad,

(1, -6.0, 12.0, 1)

In [31]:
class Neuron:
    def __init__(self, nin: int):
        self.w = [Value(random.uniform(-1, 1)) for _ in range(nin)]
        self.b = Value(random.uniform(-1, 1))

    def __call__(self, x):
        act = sum(xi*wi for xi, wi in zip(x, self.w)) + self.b
        out = act.tanh()
        return out

In [48]:
n = Neuron(2)
x = [Value(3.), Value(2.)]
n(x)

Value(-0.9969993984247137)